C'est ici que toute la documentation du pipeline, les résultats et les analyses visuelles sont intégrés.

Ce notebook contient les explications et appelle les scripts du dossier src/ pour exécuter chaque étape du pipeline.

On l'utilise pour charger et exécuter le pipeline étape par étape. 

Par exemple, pour exécuter la partie nettoyage, importer et exécuter data_preprocessing.py depuis src/.

À chaque étape (nettoyage, analyse des données, visualisations), ajouter des cellules Markdown dans le notebook pour expliquer ce que fait chaque étape, pourquoi elle est nécessaire, et les observations tirées des résultats.

C’est également ici que nous ajouterons les graphiques et analyses visuelles pour expliquer nos résultats intermédiaires.

Expliquer ce que l'on fait dans notre projet et que pour ce faire nous utiliserons une version clean de movie_metadata.
On affiche à partir de quelle année notre base de données de films commence pour exclure les évènements historiques trop lointains dans notre dataset additionnel.
On veut garder la première colonne pour relier le film à son résumé mais aussi la plupart des autres.
On veut transformer les formats de dates pour l’analyse temporelle.

In [ ]:
from src.data_preprocessing import clean_movie_data 
from tests.test_data_preprocessing import test_cleaned_movie_data
import pandas as pd

movies_data = pd.read_csv("data/MovieSummaries/movie.metadata.tsv", delimiter='\t') # Columns are separated by tabs rather than commas

# Cleaning the raw data
cleaned_movies_data = clean_movie_data(movies_data)

#Checking if the cleaning worked 
test_cleaned_movie_data(cleaned_movies_data)

# Print the cleaned data results
print(cleaned_movies_data.head())

On veut aussi clean et tokeniser les résumés de films pour en trouver plus facilement le thème (lien historique ou non) et se débarasser des valeurs manquantes.

In [ ]:
from src.data_preprocessing import vectorize_plot_data, clean_plot_data
from tests.test_data_preprocessing import test_cleaned_plot_data, test_vectorized_plot_data
import pandas as pd

plot_data = pd.read_csv("data/MovieSummaries/plot.summaries.txt", delimiter='\t', header=None, names=['id', 'plot']) #document texte

# Cleaning the raw data and checking if it worked
cleaned_plot_data= clean_plot_data(plot_data)
test_cleaned_plot_data(cleaned_plot_data)

#Vectorizing/Tokenizing the cleaned data and checking if it worked
vectorized_plot_data = vectorize_plot_data(cleaned_plot_data)
test_vectorized_plot_data(vectorize_plot_data)

# Print the cleaned and vectorized/tokenized data results
print(vectorized_plot_data.head())

Enfin, on veut clean notre dataset additionnel, création de catégories utiles pour l’analyse, comme « guerre », « crise économique », « période de stabilité ». On cherche encore ici à vectoriser et mettre sous forme de token certains types de mots en identifiant les catégories qui nous intéressent.
On veut éliminer les valeurs manquantes ainsi que les événements historiques beaucoup trop vieux pour notre analyse.
Peu importe la méthode utilisée pour tokeniser/vectroriser, expliquer ici pourquoi c'est approprié pour nos objectif et donner une  brève explication mathématique si possible.
On veut aussi transformer les formats de dates pour l’analyse temporelle à l'image de notre movie dataset.

In [ ]:
from src.data_preprocessing import vectorize_historical_data, clean_historical_data
from tests.test_data_preprocessing import test_cleaned_historical_data, test_vectorized_historical_data
import pandas as pd

historical_data = pd.read_csv("data/HistoricalDataset/historical.dataset.csv")

# Cleaning the raw data and checking if it worked
cleaned_historical_data= clean_historical_data(historical_data)
test_cleaned_historical_data(cleaned_historical_data)

#Vectorizing/Tokenizing the cleaned data and checking if it worked
vectorized_historical_data = vectorize_historical_data(cleaned_historical_data)
test_vectorized_historical_data(vectorize_historical_data)

# Print the cleaned and vectorized/tokenized data results
print(vectorized_historical_data.head())

La prochaine étape est de faire des plots, voici mes idées mais rajoutez en à votre guise!! 

- Proportion de chaque genre dans les films sortis chaque année. (lineplot?)
- Heatmap temporelle avec les années et les genres pourrait aussi aider à voir les pics et baisses de popularité des genres.
- Comparer les genres populaires à des périodes historiques précises (avant, pendant, après).
- Ligne temporelle des revenus du box office pour des genres particuliers, avec des annotations pour les films les plus réussis. Les pics ou succès importants peuvent être marqués pour voir si d'autres films de genre similaire ont suivi. (matplotib.pyplot)
- Montrez la durée pendant laquelle un genre reste populaire après le succès d’un film spécifique. Chaque barre représente un genre et montre la période pendant laquelle il est resté en tendance. (barplot?)
- Carte géographique et graphique pour montrer l’origine des films (par pays) et un graphique en barres empilées pour voir comment les genres évoluent dans les différentes régions, en mettant en avant la diffusion de genres américains. (plotly.express.choropleth et seaborn)


In [1]:
# Appels au méthodes de plot dans analysis dans src apres nettoyage.